In [ ]:
import os
import pandas as pd
import lyricsgenius as lg
import spotipy
import time

#### All tokens

In [ ]:
spotipy_client_id = 'c3e43077142f4e7bb70601747bd1f0d2'
spotipy_client_secret = '925f7025c8e6420f9be665401784c0f6'
spotipy_redirect_uri = 'https://google.com'
genius_access_token = 'jVHnDFkBvrvEsj_M8YUxbQ9t8mo2zIG19giBxlX_g7NeRz4W51gedM1YHTBX_U-4'

#### Fetching the track data

In [ ]:
from spotipy.oauth2 import SpotifyClientCredentials

client_credentials_manager = SpotifyClientCredentials (spotipy_client_id, spotipy_client_secret)
sp=spotipy.Spotify(client_credentials_manager=client_credentials_manager)

# Function to extract all the track ids from your playlist
def get_track_ids(playlist_id):
  music_id_list = []
  playlist = sp.playlist(playlist_id)
  for item in playlist['tracks']['items']:
    music_track = item['track']
    music_id_list.append(music_track['id'])
  return music_id_list

# Function to extract all the details of each track by passing its id
def get_track_data(track_id):
  meta = sp.track(track_id)
  track_details = {"name": meta['name'], "album": meta['album']['name'],
  "artist": meta['album']['artists'][0]['name']}

  return track_details

#### Fetching details of songs in given playlist

In [ ]:
# Get the ids for all the songs in your playlist
playlist_id = input('Enter the playlist id')
track_ids = get_track_ids (playlist_id)

# loop over track ids and get their data points
tracks = []
for i in range (len (track_ids)):
  time.sleep(.5)
  track = get_track_data(track_ids[i])
  tracks.append(track)


#### Forming dataframe of track data

In [ ]:
df1 = pd.DataFrame.from_dict(tracks)

#### Using genius api for lyrics

In [ ]:
genius = lg.Genius(genius_access_token)

In [ ]:
l=[]
for i in range(len(df1['name'])):
  song = genius.search_song(title=df1['name'][i],artist=df1['artist'][i])
  lyrics = song.lyrics
  if song != None:
    l.append(lyrics)

lyrics_df = pd.DataFrame()
lyrics_df['lyrics'] = l

#### Joining both Data frames

In [ ]:
# join both data frames
df = pd.concat([df1, lyrics_df], axis=1, join='inner')
df